In [1]:
%matplotlib inline

from sklearn import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#関数の引数を調べる
import inspect
#inspect.signature(f)
#バージョンの確認
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version
#Version(sklearn_version)

In [2]:
import pyprind

In [3]:
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
#顔文字は残し余計な表現は捨て、さらにstopwordsに含まれるものも捨て、空白でトークン化
def tokenizer(text):
    text= re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|d|p)',text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [4]:
#各行の文章と正解データを抽出
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) #ヘッダーを読み飛ばす
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [5]:
#文章と正解が入ったタプルからsizeのぶんだけ抜き出す
def get_minibatch(doc_stream, size):
    docs, y = [], []
    #try,exceptはtryを実行してエラーが出たらexceptを実行する
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [6]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
#decode_errorはエンコーディングの文字列と違ったものが含まれた時にどうするか
#n_featuresはハッシュ値の個数
vect = HashingVectorizer(decode_error='ignore',
                        n_features = 2**21,
                        preprocessor=None,
                        tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [7]:
pbar = pyprind.ProgBar(45)
classes = np.array([0,1])
#X_trainが空の時はNoneを返すようにget_minibatchで設定済
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:44


In [8]:
X_test, y_test = get_minibatch(doc_stream,size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.867


In [9]:
clf = clf.partial_fit(X_test, y_test)

In [11]:
#pickleはオブジェクトをシリアライズ化するモジュール
import pickle
import os
dest = os.path.join('movieclassifier', 'pkl_objects')
#このパスが存在しなかったら作る
if not os.path.exists(dest):
    os.makedirs(dest)
#pickle.dump関数でシリアライズ化
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

In [12]:
dest

'movieclassifier/pkl_objects'

In [13]:
os.path.exists(dest)

True

In [5]:
%%writefile movieclassifier/vectorizer.py
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

#今実行しているスクリプトの相対パスを求める
cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(os.path.join(cur_dir, 'pkl_objects', 'stopwords.pkl'),'rb'))

def tokenizer(text):
    text= re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|d|p)',text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',n_features=2**21, preprocessor=None, tokenizer=tokenizer)

Overwriting movieclassifier/vectorizer.py


In [20]:
import os
os.getcwd()

'/Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier'

In [7]:
#作業ディレクトリの変更
os.chdir('movieclassifier')

In [9]:
import pickle
import re
import os
from vectorizer import vect
#デシリアライズ化
clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

In [10]:
import numpy as np
label = {0: 'negative', 1:'positive'}
example = ['I love this movie']
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' %\
     (label[clf.predict(X)[0]], (clf.predict_proba(X)).max()*100))

Prediction: positive
Probability: 82.52%


SQLite3でデータベースを作る

In [1]:
import sqlite3 
conn = sqlite3.connect('review.sqlite')
#cursorを作ってexecuteでSQL文を実行
c = conn.cursor()
#テーブルを作成　（）には列名、データ型が入る
c.execute("CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)")
example1 = "I love this movie"
#?,? は後ろのタプルのメンバーの位置を示す引数、
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example1, 1))
example2 = "I disliked this movie"
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example2, 0))
conn.commit()
conn.close()

In [2]:
conn = sqlite3.connect('review.sqlite')
c = conn.cursor()
#WHERE 列名　条件
c.execute("SELECT * FROM review_db WHERE date"\
         " BETWEEN '2016-01-01 00:00:00' AND DATETIME('now')")
# fetchall() メソッドを呼んで一致した全ての行のリストとして受け取る
results = c.fetchall()
conn.close()
print(results)

[('I love this movie', 1, '2016-10-16 05:31:14'), ('I disliked this movie', 0, '2016-10-16 05:31:14')]


In [6]:
import pandas as pd
pd.DataFrame(results,columns=['review', 'sentiment', 'date'])

,review,sentiment,date
0,I love this movie,1,2016-10-16 05:31:14
1,I disliked this movie,0,2016-10-16 05:31:14


In [19]:
import sqlite3 
conn2 = sqlite3.connect('review.sqlite2')
#cursorを作ってexecuteでSQL文を実行
c = conn2.cursor()
#テーブルを作成　（）には列名、データ型が入る
c.execute("CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)")
example1 = "I love this movie"
#?,? は後ろのタプルのメンバーの位置を示す引数、
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (example1, 1, DATETIME('now'))")
example2 = "I disliked this movie"
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))",(example2, 0))
conn2.commit()
conn2.close()

OperationalError: no such column: example1

In [24]:
os.getcwd()

'/Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier'

最初のアプリケーション

In [29]:
%%writefile /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_1/app.py

from flask import Flask, render_template

#新しいFlaskインスタンスを__name__という引数で初期化
#HTMLテンプレートフォルダがこのモジュールと同じフォルダで見つかることをFlaskに教えている
app = Flask(__name__)

#@app.route('/)はルートデコレータと呼ばれ、index関数の実行を開始すべきURLを指定している
@app.route('/')
#index関数はfirst_app.htmlをレンダリングするだけ
def index():
    return render_template('first_app.html')

#スクリプトファイルとして実行された時のみ実行
#モジュールとしてimportで実行された時には実行されない
if __name__ == '__main__':
    app.run()

Overwriting /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_1/app.py


In [26]:
%%writefile /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_1/templates/first_app.html

<!doctype html>
<html>
    <head>
        <title>First App</title>
    </head>
    <body>
        #<div>は特に意味はないがひとかたまりであることを示す
        <div>Hi, this is my first Flask web app!</div>
    </body>
</html>

Writing /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_1/templates/first_app.html


二個目のアプリケーション

In [30]:
%%writefile /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_2/app.py

from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators

#新しいFlaskインスタンスを__name__という引数で初期化
#HTMLテンプレートフォルダがこのモジュールと同じフォルダで見つかることをFlaskに教えている
#この一つのモジュールしか使わないなら、__name__でいつもOK
#もしパッケージの中で使うならパッケージの名前を入れる
app = Flask(__name__)

#ステートメントの引数は、「継承するクラス」。継承するクラスがない場合はobject

class HelloFrom(Form):
    sayhello = TextAreaField('',[validators.DataRequired()] )

#@app.route('/)はルートデコレータと呼ばれ、index関数の実行を開始すべきURLを指定している
# route() デコレータは ファンクションとURLをバインドするために使われる
@app.route('/')
#index関数はfirst_app.htmlをレンダリングするだけ
def index():
    form = HelloFrom(request.form)
    return render_template('first_app.html', form=form)

@app.route('/hello', methods=['POST'])
def hello():
    form = HelloFrom(request.form)
    if request.method == 'POST' and form.validate():
        name = request.form['sayhello']
        return render_template('hello.html', name=name)
    return render_template('first_app.html', form=form)

#スクリプトファイルとして実行された時のみ実行
#モジュールとしてimportで実行された時には実行されない
if __name__ == '__main__':
    #debug=Trueにするとコード変更時に自動でリロードしてくれる
    app.run(debug=True)

Writing /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_2/app.py


In [34]:
from wtforms import Form, fields, validators

class TestForm(Form):
    hoge = fields.TextAreaField('', [validators.DataRequired()])

In [35]:
print(TestForm().hoge) #<textarea id="hoge" name="hoge"></textarea>

<textarea id="hoge" name="hoge"></textarea>


In [36]:
%%writefile /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_2/templates/_formhelpers.html
{% macro render_field(field) %}
  <dt>{{ field.label }}
  <dd>{{ field(**kwargs)|safe }}
  {% if field.errors %}
    <ul class=errors>
    {% for error in field.errors %}
      <li>{{ error }}</li>
    {% endfor %}
    </ul>
  {% endif %}
  </dd>
  </dt>
{% endmacro %}

Writing /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_2/templates/_formhelpers.html


In [37]:
%%writefile /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_2/static/style.css
body {
    font-size: 2em;
}

Writing /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_2/static/style.css


In [38]:
%%writefile /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_2/templates/first_app.html
<!doctype html>
<html>
  <head>
    <title>First app</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  </head>
  <body>

{% from "_formhelpers.html" import render_field %}

<div>What's your name?</div>
<form method=post action="/hello">

  <dl>
      {{ render_field(form.sayhello) }}
  </dl>

  <input type=submit value='Say Hello' name='submit_btn'>

</form>

  </body>
</html>

Writing /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_2/templates/first_app.html


In [39]:
%%writefile /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_2/templates/hello.html
<!doctype html>
<html>
  <head>
    <title>First app</title>
	<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  </head>
  <body>

<div>Hello {{ name }}</div>


  </body>
</html>

Writing /Users/keeeeei/Desktop/データ分析/Python機械学習プログラミング/movieclassifier/1st_flask_app_2/templates/hello.html


In [8]:
input = ['1', '2','3']

In [9]:
int(input)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [ ]:
import numpy as np
n, m, k = input().rstrip().split(' ')
n, m ,k = int(n), int(m), int(k) 
gen = [int(x) for x in input_lines] 
num = int(input())
for i in range(num):
    lis = input().rstrip().split(' ')
    lis = [int(x) for x in lis] 
    if ((lis[0]-gen[0])**2 + (lis[1]-gen[1])**2) >= gen[2]**2:
        print('silent')
    else:
        print('noisy')

In [60]:
import numpy as np
lis = [1,2,3,4,5,8,1,1,1,8,8,-8,5,5]
#np.sum([lis[x] for x in [3,5,2]] )
a = []
for x in lis:
    if x >= 0:
        a.append(x)
    else:
        a.append(0)
np.sum(a)
    


52

In [62]:
a = []
a[0] = 1

IndexError: list assignment index out of range

In [63]:
[0]*2

[0, 0]

In [48]:
a[0] in lis

True

In [50]:
for ind, x in enumerate(lis):
    lis[ind] = 0
lis

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [20]:
[int(x[0]) for x in list(map(str, lis))]

[1, 2, 3, 4, 5, 8]

In [ ]:
nl = [i for i in range(15)] 
ev = nl[0::2]
od = nl[1::2]
guu = [lis[x] for x in ev]
evn = [x*2 for x in guu]
val = map(str, evn)
even = [np.where(len(x)>=2, int(x[0])+int(x[1]) , int(x)) for x in val]
even = np.sum(even)
odd = np.sum([lis[x] for x in od])
X = 10 - ((even + odd) % 10)
if X == 10:
    X = 0
print(X)
    

In [23]:
len('12')

2

In [35]:
lis = [21,47]
#lis = map(str,[1,2,3,4,5,48,1,21,1,8,8,8,5,5])
[np.where(len(a)>=2.0, int(a[0])+int(a[1]), int(a)) for a in lis]

TypeError: object of type 'int' has no len()

In [34]:
[a for a in lis]

['2', '3', '4', '5', '48', '1', '21', '1', '8', '8', '8', '5', '5']

In [36]:
12.5 % 10

2.5

In [ ]:
# coding: utf-8
# 自分の得意な言語で
# Let's チャレンジ！！
import numpy as np
n, m, k = input().rstrip().split(' ')
n, m ,k = int(n), int(m), int(k)
lis = []
ans = []
for i in range(m):
    l = int(input())
    lis.append(l)
for _ in range(k):
    for ind, x in enumerate(lis):
        while x in lis:
            x += 1
            if x > n:
                x = 1
        lis[ind] = x
for i in lis:
    print(i)

In [71]:
import numpy as np
np.zeros(1) == 0

array([ True], dtype=bool)

In [80]:
[0::1]

SyntaxError: invalid syntax (<ipython-input-80-e537b50ce7ff>, line 1)